In [6]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
try:
    from . import contants
except ImportError:
    import constants

import re
    
from humanfriendly import prompts
import textwrap
import logging



In [4]:
from library import GDrive

In [ ]:
d = GDrive(secrets='./secrets/client_secret_910311278281-bh8qk3kmgk0veri3v8en260e76ipafpj.apps.googleusercontent.com.json', scopes=constants.SCOPES)

In [ ]:
d.search('Misc Junk')

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [ ]:
def do_exit(exit_level=0, message='', testing=False):

    if testing:
        print(f'TESTING--EXIT MESSAGE: ({exit_level}){message}')
    elif exit_level > 0:
        logger.warning(f'exiting before completion with exit code {exit_level}: {message}')
        print(message)
        exit(exit_level)
    else:
        print(message)
        exit(0)
        

In [ ]:
def recurse_folders(drive, parents, 
                    fields=['parents', 'id', 'name', 'mimeType', 'owners', 'modifiedTime', 'webViewLink', 'parents'], 
                    file_list=[], skipped=[], depth=0):
    '''recursively find all files in a google drive folder'''
    if depth == 0:
        file_list = []
        skipped = []
    logger.info(f'recursion depth: {depth}')
    try:
        result = drive.search(parents=parents, fields=fields)
    except GDriveError as e:
        logger.error(f'error accessing google drive: {e}')
        skipped.append(parents)
        result = {}
        
    for f in result:
        if drive.MIMETYPES['folder'] == f.get('mimeType'):
            return_files, return_skipped = recurse_folders(drive=drive, parents=f['id'], 
                                         fields=fields, 
                                         file_list=file_list,
                                         skipped=skipped,
                                         depth=depth+1)
            file_list + return_files
            skipped + return_files
        else:
            file_list.append(f)
    return (file_list, skipped)
            

In [ ]:
def get_folder_id():
    '''get folder id from URL provided by user
    
    Returns:
        str: unique google drive folder id'''
    valid_url = False
    retry = 4
    folderID = None
    print('\n')
    while not valid_url and retry > 0:
        retry -= 1
        url = prompts.prompt_for_input('Paste the URL of a Google Drive folder to audit:\n')
        
        match = re.match('https:\/\/drive.google.com\/.*(?:\w+\/|=)([a-zA-Z0-9_-]+)', url)
        if not match:
            print('That does not appear to be a valid google drive folder URL')
            continue
        else:
            folderID = match[1]
            valid_url = True
    
    return folderID

In [ ]:
def main():
    
    
    
    drive = GDrive(secrets='./secrets/client_secret_910311278281-bh8qk3kmgk0veri3v8en260e76ipafpj.apps.googleusercontent.com.json', scopes=constants.SCOPES)
    
    wrapper = textwrap.TextWrapper(replace_whitespace=True, drop_whitespace=True, width=60)
    welcome = f'{constants.APP_NAME} provides a Google Sheet that shows all of the files and owners within a google drive folder.'
    print('\n'.join(wrapper.wrap(welcome)))
    folder_id = get_folder_id()
    
    if not folder_id:
        do_exit(1, 'No valid folder URL was provided. Exiting')
        
    


In [19]:
def get_secrets():
    if not constants.SECRETS_PATHS['saved'].exists():
        constants.SECRETS_PATHS['saved'].parent.mkdir(parents=True, exist_ok=True)
        
    if not constants.SECRETS_PATHS['saved'].exists():
        ## figure out how to do this, provid instructions when it fails
        print(f"copy {constants.SECRETS_PATHS['downloaded']} to {constants.SECRETS_PATHS['saved']}")


In [17]:
constants.SECRETS_PATHS['saved'].exists()

False

In [20]:
get_secrets()

copy /Users/aciuffo/Downloads/client_secrets.json to /Users/aciuffo/.cache/gdrive_audit/client_secrets.json


In [ ]:
main()

In [ ]:
wrapper = textwrap.TextWrapper(replace_whitespace=True, drop_whitespace=True, width=45)